In [2]:
import pandas as pd
import sys
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from tsfresh.feature_extraction import MinimalFCParameters


import tsfresh
from tsfresh.feature_extraction import extract_features

from xgboost import XGBRegressor

In [3]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [4]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCounter
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_by_index_transformer import SelectByIndexTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_by_mask_transformer import SelectByMaskTransformer
from transformers.select_features_wrapper import SelectFeaturesWrapper
from transformers.my_turns_transformation import MyTurnsTransformation
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_by_index_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_by_mask_transformer)
importlib.reload(transformers.select_features_wrapper)
importlib.reload(transformers.my_turns_transformation)


import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

train model

In [5]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')


In [6]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

features = np.load('../feature_selection_consts/columns.npy', allow_pickle=True)


prePipe = Pipeline([
                    ('get_relavent_turns', SelectByIndexTransformer(train.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectByIndexTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectByMaskTransformer('is_player', target=DATA_NAME)),
                    ])
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
t_turns = MyTurnsTransformation().transform(n_turns) 
n_data = n_games.merge(t_turns, left_index=True, right_index=True)



Feature Extraction: 100%|██████████| 20/20 [02:20<00:00,  7.00s/it]


Pipeline(steps=[('extract_first',
                 ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x00000271FBFDB4F0>})),
                ('hot',
                 OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                'STEEBot',
                                                                'HastyBot'],
                                                   'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=object),
                                                   'lexicon': array(['...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.17320508075688773, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=7, max_leaves=None,
                              min_child_weight=1, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=0, ...))])

In [30]:
regressor_steps = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('select_col', SelectTransformer(features)),
                       ('drop_bot_rating', NameDropperTransformer(['bot_rating'])),
                       ('xgb', XGBRegressor(random_state=0,
                              max_depth=7, min_child_weight=1,
                              gamma=0,
                              subsample=1, colsample_bytree=1,
                              reg_alpha=0.00005, reg_lambda=1,
                              learning_rate=np.sqrt(3)/10
                              ),)                                           
                      ]

regressor = Pipeline(regressor_steps)
regressor.fit(n_data, n_ratings)

Pipeline(steps=[('extract_first',
                 ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x00000271C3EFA4F0>})),
                ('hot',
                 OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                'STEEBot',
                                                                'HastyBot'],
                                                   'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=object),
                                                   'lexicon': array(['...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.17320508075688773, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=7, max_leaves=None,
                              min_child_weight=1, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=0, ...))])

predict:

In [8]:
test = pd.read_csv('../data/test.csv', index_col='game_id')

In [12]:
test['rating'].isna().sum()

22363

In [18]:
test_pre_pipe = Pipeline([
                    ('get_relavent_turns', SelectByIndexTransformer(test.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectByIndexTransformer(test.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectByMaskTransformer('is_player', target=DATA_NAME)),
                    ])
test_preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, test_pre_pipe).build()
test_n_games, test_n_turns, test_n_ratings = test_preprocessor.process(test, DATA_NAME)
test_t_turns = MyTurnsTransformation().transform(test_n_turns) 
test_n_data = test_n_games.merge(test_t_turns, left_index=True, right_index=True)

Feature Extraction: 100%|██████████| 20/20 [00:52<00:00,  2.61s/it]


In [31]:
prediction = pd.Series(regressor.predict(test_n_data), index=test_n_data.index, name='rating')

In [32]:
prediction.to_csv('../data/submission.csv', index_label='game_id')